In [4]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

import matplotlib.pyplot as plt

In [5]:
WORKING_DIR = "/Users/junyangwu/Junyang Wu/Study/Columbia/2025.9-2025.12/4004 Optimization model and methods/Group Study/Project 2"

In [6]:
df = pd.read_csv(f'{WORKING_DIR}/games.csv')
df

,Date,Visitor,PTS,Home,PTS.1,Attend.,LOG,Arena,Notes
0,"Sat, Nov 01, 2025",Golden State Warriors,NaN,Boston Celtics,NaN,"19,000",7:30 PM,TD Garden,NaN
1,"Sat, Nov 01, 2025",Los Angeles Lakers,NaN,New York Knicks,NaN,"19,400",7:30 PM,Madison Square Garden,NaN
2,"Sat, Nov 01, 2025",Denver Nuggets,NaN,Brooklyn Nets,NaN,"17,500",7:30 PM,Barclays Center,NaN
3,"Sat, Nov 01, 2025",Phoenix Suns,NaN,Philadelphia 76ers,NaN,"19,650",7:30 PM,Wells Fargo Center,NaN
4,"Sat, Nov 01, 2025",Houston Rockets,NaN,Toronto Raptors,NaN,"19,600",7:30 PM,Scotiabank Arena,NaN
...,...,...,...,...,...,...,...,...,...
123,"Thu, Dec 25, 2025",Philadelphia 76ers,NaN,Phoenix Suns,NaN,"17,450",7:30 PM,Footprint Center,Christmas Day
124,"Thu, Dec 25, 2025",Toronto Raptors,NaN,Houston Rockets,NaN,"17,800",7:30 PM,Toyota Center,Christmas Day
125,"Thu, Dec 25, 2025",Miami Heat,NaN,Dallas Mavericks,NaN,"19,800",7:30 PM,American Airlines Center,Christmas Day
126,"Thu, Dec 25, 2025",Chicago Bulls,NaN,Atlanta Hawks,NaN,"17,250",7:30 PM,State Farm Arena,Christmas Day


In [7]:
teams = df['Home'].unique().tolist()
teams.sort()
teams

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Golden State Warriors',
 'Houston Rockets',
 'Los Angeles Lakers',
 'Miami Heat',
 'Milwaukee Bucks',
 'New York Knicks',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Toronto Raptors']

# Question 1

In [8]:
df = df.rename(columns={"Date": "date","Visitor": "visitor","Home": "home",})
df = df[["date", "visitor", "home"]].copy()

df["date_parsed"] = pd.to_datetime(df["date"], errors="coerce")
df["date_final"] = df["date_parsed"].dt.date.astype("string")
null_mask = df["date_parsed"].isna()
df.loc[null_mask, "date_final"] = df.loc[null_mask, "date"]

## (a)

In [9]:
home_dates_series = (df.groupby("home")["date_final"].agg(list).sort_index())
home_dates_df = (home_dates_series.apply(lambda x: ", ".join([d for d in x if isinstance(d, str)])).reset_index().rename(columns={"home": "team", "date_final": "home_dates"}))
home_dates_df

,team,home_dates
0,Atlanta Hawks,"2025-11-03, 2025-11-07, 2025-11-15, 2025-11-17..."
1,Boston Celtics,"2025-11-01, 2025-11-07, 2025-11-13, 2025-11-19..."
2,Brooklyn Nets,"2025-11-01, 2025-11-05, 2025-11-11, 2025-11-13..."
3,Chicago Bulls,"2025-11-01, 2025-11-03, 2025-11-13, 2025-11-21..."
4,Cleveland Cavaliers,"2025-11-03, 2025-11-07, 2025-11-17, 2025-11-19..."
5,Dallas Mavericks,"2025-11-03, 2025-11-07, 2025-11-15, 2025-11-17..."
6,Denver Nuggets,"2025-11-05, 2025-11-07, 2025-11-11, 2025-11-15..."
7,Golden State Warriors,"2025-11-03, 2025-11-05, 2025-11-11, 2025-11-15..."
8,Houston Rockets,"2025-11-03, 2025-11-11, 2025-11-13, 2025-11-15..."
9,Los Angeles Lakers,"2025-11-03, 2025-11-05, 2025-11-11, 2025-11-15..."


## (b)

In [10]:
home_vs_counts = pd.crosstab(index=df["home"], columns=df["visitor"]).sort_index().sort_index(axis=1)
home_vs_counts

visitor,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Lakers,Miami Heat,Milwaukee Bucks,New York Knicks,Philadelphia 76ers,Phoenix Suns,Toronto Raptors
home,,,,,,,,,,,,,,,,
Atlanta Hawks,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,1
Boston Celtics,0,0,0,0,0,1,0,1,0,0,1,1,0,1,1,0
Brooklyn Nets,1,1,0,0,0,1,1,1,0,0,1,0,0,1,0,0
Chicago Bulls,1,1,1,0,0,0,1,1,1,1,0,1,0,0,0,1
Cleveland Cavaliers,1,1,1,1,0,0,1,0,1,0,1,1,1,0,0,1
Dallas Mavericks,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1
Denver Nuggets,1,1,1,0,0,0,0,0,1,1,1,0,1,1,0,0
Golden State Warriors,0,1,0,0,1,0,1,0,1,0,0,1,1,1,0,0
Houston Rockets,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1


## (c)

In [11]:
away_vs_counts = pd.crosstab(index=df["visitor"], columns=df["home"]).sort_index().sort_index(axis=1)
away_vs_counts

home,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Golden State Warriors,Houston Rockets,Los Angeles Lakers,Miami Heat,Milwaukee Bucks,New York Knicks,Philadelphia 76ers,Phoenix Suns,Toronto Raptors
visitor,,,,,,,,,,,,,,,,
Atlanta Hawks,0,0,1,1,1,0,1,0,1,0,0,0,0,1,0,0
Boston Celtics,1,0,1,1,1,0,1,1,1,1,0,0,1,0,0,1
Brooklyn Nets,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,1
Chicago Bulls,1,0,0,0,1,1,0,0,0,0,1,0,1,1,1,0
Cleveland Cavaliers,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0
Dallas Mavericks,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0
Denver Nuggets,0,0,1,1,1,1,0,1,0,0,0,1,0,0,1,1
Golden State Warriors,1,1,1,1,0,1,0,0,0,1,1,0,0,0,1,1
Houston Rockets,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1


## (d)

In [12]:
away_dates_series = (df.groupby("visitor")["date_final"].agg(list).sort_index())
away_dates_df = (away_dates_series.apply(lambda x: ", ".join([d for d in x if isinstance(d, str)])).reset_index().rename(columns={"visitor": "team", "date_final": "away_dates"}))
away_dates_df

,team,away_dates
0,Atlanta Hawks,"2025-11-01, 2025-11-05, 2025-11-11, 2025-11-13..."
1,Boston Celtics,"2025-11-03, 2025-11-05, 2025-11-11, 2025-11-15..."
2,Brooklyn Nets,"2025-11-03, 2025-11-07, 2025-11-15, 2025-11-17..."
3,Chicago Bulls,"2025-11-05, 2025-11-07, 2025-11-11, 2025-11-15..."
4,Cleveland Cavaliers,"2025-11-01, 2025-11-05, 2025-11-11, 2025-11-13..."
5,Dallas Mavericks,"2025-11-01, 2025-11-05, 2025-11-11, 2025-11-13..."
6,Denver Nuggets,"2025-11-01, 2025-11-03, 2025-11-13, 2025-11-23..."
7,Golden State Warriors,"2025-11-01, 2025-11-07, 2025-11-13, 2025-11-19..."
8,Houston Rockets,"2025-11-01, 2025-11-05, 2025-11-07, 2025-11-17..."
9,Los Angeles Lakers,"2025-11-01, 2025-11-07, 2025-11-13, 2025-11-21..."


In [13]:
export_path = f"{WORKING_DIR}/export_data/output_summary.xlsx"
with pd.ExcelWriter(export_path, engine="openpyxl") as writer:
    home_dates_df.to_excel(writer, sheet_name="(a)home_dates", index=False)
    home_vs_counts.to_excel(writer, sheet_name="(b)home_vs_counts")
    away_vs_counts.to_excel(writer, sheet_name="(c)away_vs_counts")
    away_dates_df.to_excel(writer, sheet_name="(d)away_dates", index=False)

# Question 2

Notations:

- Set of teams: $T$
- Set of dates: $D$

For each team $i \in T$:

- $H_i \subseteq D$: dates on which team $i$ must play **at home**
- $A_i \subseteq D$: dates on which team $i$ must play **away**

Parameters:

- $h_{i,t} = 1$ if $t \in H_i$, $0$ otherwise  
- $a_{i,t} = 1$ if $t \in A_i$, $0$ otherwise  
- $b_{ij}$: number of times team $i$ plays home vs. team $j$  
- $c_{ij}$: number of times team $i$ plays away vs. team $j$ (at $j$'s home)

Decision variable

$$
x_{ijt} =
\begin{cases}
1, & \text{if team } i \text{ plays at home against } j \text{ on date } t, \\
0, & \text{otherwise}
\end{cases}
\qquad (i \neq j,\; t \in D)
$$


## (e)
For every team $i$ and every date $t$, team $i$ plays at home on date $t$ if and only if
$t$ is one of its home dates $H_i$:

$$
\sum_{j \in T,\, j \neq i} x_{ijt} = h_{i,t}
\qquad \forall i \in T,\; \forall t \in D.
$$

---

## (f)

For every team $i$ and every date $t$, team $i$ plays away on date $t$ if and only if
$t$ is one of its away dates $A_i$. Being away means it is the visiting team in some
home game of another team:

$$
\sum_{j \in T,\, j \neq i} x_{jit} = a_{i,t}
\qquad \forall i \in T,\; \forall t \in D.
$$

---

## (g)

For every ordered pair of distinct teams $(i,j)$, the total number of home games
where $i$ hosts $j$ must equal $b_{ij}$:

$$
\sum_{t \in D} x_{ijt} = b_{ij}
\qquad \forall i,j \in T,\; i \neq j.
$$

---

## (h)

For every ordered pair of distinct teams $(i,j)$, the total number of away games
where $i$ visits $j$ (i.e., $j$ is at home versus $i$) must equal $c_{ij}$

$$
\sum_{t \in D} x_{jit} = c_{ij}
\qquad \forall i,j \in T,\; i \neq j.
$$
